<a href="https://colab.research.google.com/github/fabibo3/music-genre-classification/blob/main/compute_vgg_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install soundfile

In [2]:
# Clone repo
!rm -r music-genre-classification/
!git clone https://github.com/fabibo3/music-genre-classification.git

rm: cannot remove 'music-genre-classification/': No such file or directory
Cloning into 'music-genre-classification'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 289 (delta 168), reused 187 (delta 90), pack-reused 0
Receiving objects: 100% (289/289), 87.92 KiB | 343.00 KiB/s, done.
Resolving deltas: 100% (168/168), done.


In [3]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import librosa, librosa.display
import matplotlib.pyplot as plt
import soundfile as sf
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import os
import pickle
from tqdm import tqdm
from audioread.exceptions import NoBackendError

# Load the model.
vggmodel = hub.load('https://tfhub.dev/google/vggish/1')

def embedding_from_fn(fn):
  x, sr = librosa.load(fn) #,sr=None
  x_16k = librosa.resample(x,sr,16000) #resample to 16KHz
  embedding = np.array(vggmodel(x_16k)) 
  return embedding


In [ ]:
# Train data

dataset = []
valid_ids = []
not_valid = []

file_dir = "/content/drive/MyDrive/data/train/Train/"
files = os.listdir(file_dir)
files = sorted(files)
for fn in tqdm(files):
    fn_full = os.path.join(file_dir, fn)
    try:
      emb = embedding_from_fn(fn_full)
      dataset.append(emb[0:295]) # limit if required
      #print('taille embed : ', emb.shape)
      valid_ids.append(int(fn.split(".")[0]))
    except NoBackendError:
      not_valid.append(fn)

print("Files not valid:", not_valid)

# Pickle
valid_ids = np.asarray(valid_ids)
dataset = np.asarray(dataset)
id_file_name = "/content/drive/MyDrive/data/train/preprocessed/vgg_train_ids_new.pickle"
data_file_name = "/content/drive/MyDrive/data/train/preprocessed/vgg_train_new.pickle"
pickle.dump(valid_ids, open(id_file_name, "wb"))
pickle.dump(dataset, open(data_file_name, "wb"))

  0%|          | 12/4000 [00:46<4:40:57,  4.23s/it]

In [ ]:
# Check
ids = pickle.load(open(id_file_name, "rb"))
print(ids.shape)
print(ids[0])

data = pickle.load(open(data_file_name, "rb"))
print(data.shape)
print(data[0])



In [ ]:
#Test data

dataset = []
valid_ids = []
not_valid = []

file_dir = "/content/drive/MyDrive/data/test/Test/"
files = os.listdir(file_dir)
files = sorted(files)
for fn in tqdm(files[:5]):
    fn_full = os.path.join(file_dir, fn)
    try:
      emb = embedding_from_fn(fn_full)
      dataset.append(emb[0:295]) # limit if required
      #print('taille embed : ', emb.shape)
      valid_ids.append(int(fn.split(".")[0]))
    except NoBackendError:
      not_valid.append(fn)

print("Files not valid:", not_valid)

# Pickle
valid_ids = np.asarray(valid_ids)
dataset = np.asarray(dataset)
id_file_name = "/content/drive/MyDrive/data/test/preprocessed/vgg_test_ids_new.pickle"
data_file_name = "/content/drive/MyDrive/data/test/preprocessed/vgg_test_new.pickle"
pickle.dump(valid_ids, open(id_file_name, "wb"))
pickle.dump(dataset, open(data_file_name, "wb"))

In [ ]:
# Check
ids = pickle.load(open(id_file_name, "rb"))
print(ids.shape)
print(ids[0])

data = pickle.load(open(data_file_name, "rb"))
print(data.shape)
print(data[0])
